In [1]:
from dotenv import load_dotenv
import os
from autogen import AssistantAgent, UserProxyAgent
import autogen
import faiss
from langchain_community.vectorstores import FAISS
from openai import OpenAI

In [2]:
load_dotenv('/mnt/data1tb/thangcn/datnv2/.env')
openai_key = os.getenv("OPENAI_API_KEY")

In [3]:
config_list = [
    {
        "model": "gpt-4o",
        "api_key": openai_key,
    }
]

llm_config_proxy = {
    "seed": 42,  
    "temperature": 0,
    "config_list": config_list,
    "request_timeout": 600
}

In [4]:
llm_config_assistant = {
    "seed" : 42,
    "temperature": 0,
        "functions": 
        [
            {
                "name": "answer_PDF_question",
                "description": "Answer any PDF related questions",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The question to ask in relation to PDF",
                        }
                    },
                    "required": ["question"],
                },
                
            }
        ],
    "config_list": config_list,
    "timeout": 120,
}

In [5]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config_assistant,
    system_message="""You are a helpful assistant, Answer the question 
                    based on the context. Keep the answer accurate. 
                    Respond "Unsure about answer" if not sure about 
                    the answer."""
            
)

In [6]:
from dotenv import load_dotenv
from sentence_transformers import CrossEncoder
from langchain.retrievers import EnsembleRetriever  
from langchain_community.retrievers import BM25Retriever  
from langchain_core.documents import Document  
from dotenv import load_dotenv
import os
load_dotenv()

def hybrid_search(vectorstore,query, k):
    retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": k})

    documents = [
        Document(page_content=doc.page_content, metadata=doc.metadata)
        for doc in vectorstore.similarity_search(query, k=100)
    ]
    keyword_retriever = BM25Retriever.from_documents(documents)
    keyword_retriever.k =  k

    ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],weights=[0.5, 0.5])
    return ensemble_retriever

def retrieve_and_re_rank(vector_db, query, k=10):
    cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    ensemble_search = hybrid_search(vector_db,query, k)

    docs_rel = ensemble_search.get_relevant_documents(query)

    pairs = [[query, doc] for doc in docs_rel]

    rerank_scores = cross_encoder.predict(pairs)

    ranked_docs = sorted(zip(docs_rel, rerank_scores), key=lambda x: x[1], reverse=True)
    
    results = [doc for doc, _ in ranked_docs]
    scores = [score for _, score in ranked_docs]
    
    return results, scores

/home/duyhoang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
client = OpenAI(api_key=openai_key)

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
import autogen
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="thang1943/vietnamese-bi-encoder-v2",
    model_kwargs={'device': 'cpu'}
)

/tmp/ipykernel_686668/347176074.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [10]:
vector_db = FAISS.load_local('/mnt/data1tb/thangcn/datnv2/vector_database/faiss/doctor_info',embeddings, allow_dangerous_deserialization=True)


In [11]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vector_db.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", 
    return_messages=True),
)

/tmp/ipykernel_686668/3625754763.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  OpenAI(temperature=0),
/tmp/ipykernel_686668/3625754763.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history",


In [12]:
def answer_PDF_question(question: str):
    vector_db = FAISS.load_local('/mnt/data1tb/thangcn/datnv2/vector_database/faiss/doctor_info',embeddings, allow_dangerous_deserialization=True)
    retrieved_docs , _= retrieve_and_re_rank(vector_db, question, 10)

    llm = ChatOpenAI(
        temperature=0,
        streaming=True,
        model='gpt-4o',
        api_key=openai_key
    )   

    system = """You are a helpful assistant that utilizes retrieved information"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent"),
    ])

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    response = llm_chain.run({
        "input": f"Context:\n{retrieved_docs}\n\nQuestion: {question}"
    })

    print("Answer: ", response)

In [13]:
user_proxy = autogen.UserProxyAgent(  
    name="user_proxy",
    human_input_mode="NEVER", 
    max_consecutive_auto_reply=10,
    code_execution_config={
        "use_docker": False,
    },
    # llm_config_assistant = llm_config_assistant,
    function_map={
        "answer_PDF_question": answer_PDF_question
    }
)

In [14]:
user_proxy.initiate_chat(
    assistant,
    message="""
    PGS.TS. Lê Công Định có chức vụ gì
    """
)

user_proxy (to assistant):


    PGS.TS. Lê Công Định có chức vụ gì
    

--------------------------------------------------------------------------------


OperationalError: no such column: "size" - should this be a string literal in single-quotes?

In [3]:
import os
from autogen import AssistantAgent, UserProxyAgent

llm_config = { "config_list": [{ "model": "gpt-4o", "api_key": os.environ.get("OPENAI_API_KEY") }] }
assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices.",
)

user_proxy (to assistant):

Tell me a joke about NVDA and TESLA stock prices.

--------------------------------------------------------------------------------


OperationalError: no such column: "size" - should this be a string literal in single-quotes?